In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain_teddynote import logging

logging.langsmith('app_03')
# logging.langsmith('app_03', set_enable=False)

LangSmith 추적을 시작합니다.
[프로젝트명]
app_03


<br>

# <b>CHAPTER 04 LCEL 인터페이스</b>

<br>

# <b>1. LCEL 인터페이스</b>

<br>

### LCEL 문법은 완성된 체인을 만드는 것이라고 할 수 있습니다.
<br>

#### Langchain에서는 사용자 정의 체인을 가능한 한 쉽게 만들 수 있도록, "Runnable" 프로토콜을 구현했습니다.
#### chain을 구성할 때 llm, output parser를 묶어서 구성하는데 이런 llm이나 output parser이 Runnable입니다.
#### 사용자 정의 체인은 이렇게 묶어서 만들어진 chain입니다.

<br>

#### chain을 묶을 때 구성되는 llm, output parser와 같은 각각의 모듈은 모두 Runnable 프로토콜을 가지고 있습니다. 
#### “실행 가능한 객체다.”라고 해서 Runnable이라는 표현을 사용합니다.

<br>

#### Runnable은 prompt가 될 수도 있고, llm 객체가 될 수도 있고, output parer가 될 수도 있습니다.
#### 이렇게 Runnable(llm, output parser.. 등)들을 묶어서 chain을 구성합니다.
#### 묶어서 만들어진 chain도 Runnable(실행 가능한 객체)입니다. 
#### invoke()를 호출할 수 있으면 모두 Runnable입니다.

<br>

#### 위와 같이 묶어놓은 chain1, chain2이 있다면 chain1도 Runnable이고 chain2도 Runnable입니다.
#### 두 개의 chain을 병렬적으로 실행 가능하게 만들어 주는 것이 RunnableParallel입니다.

<br>

#### Runnable 프로토콜은 대부분의 컴포넌트에 구현되어 있습니다.

<br>

#### 이는 표준 인터페이스로, 사용자 정의 체인을 정의하고 표준 방식으로 호출하는 것을 쉽게 만듭니다. 
#### 표준 인터페이스에는 다음이 포함됩니다.

#### - stream: 응답의 청크를 스트리밍합니다.
#### - invoke: 입력에 대해 체인을 호출합니다.
#### - batch: 입력 목록에 대해 체인을 호출합니다.

<br><br>

#### 비동기 메소드도 있습니다.

#### - astream: 비동기적으로 응답의 청크를 스트리밍합니다.
#### - ainvoke: 비동기적으로 입력에 대해 체인을 호출합니다.
#### - abatch: 비동기적으로 입력 목록에 대해 체인을 호출합니다.
#### - astream_log: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

<br>

# <b>2. stream: 실시간 출력<b>

<br>

### stream

In [3]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1,
    max_tokens=2048,
)

prompt = PromptTemplate.from_template("{pruduct}에 대해서 3문장으로 설명해줘.")
chain = prompt | llm | StrOutputParser()
answer = chain.stream({"pruduct": "겔럭시폰"})

# chain.stream 메소드를 사용하여 '겔럭시폰' product에 대한 스트림을 생성하고 반복합니다.
for token in answer:
    # 스트림 에서 받은 데이터의 내용을 출력합니다. 
    # 줄 바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

갤럭시폰은 삼성전자가 제조한 스마트폰 시리즈로, 다양한 모델과 기능을 제공합니다. 이 시리즈는 고급스러운 디자인과 뛰어난 카메라 성능, 그리고 강력한 성능을 자랑합니다. 또한, 안드로이드 운영체제를 기반으로 하여 사용자에게 다양한 앱과 서비스를 이용할 수 있는 환경을 제공합니다.

<br>

# <b>3. invoke: 호출</b>

<br>

### invoke

In [4]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini', 
    temperature=0.1,
    max_tokens=2048,
)

prompt = PromptTemplate.from_template("{pruduct}에 대해서 3문장으로 설명해줘.")
chain = prompt | llm | StrOutputParser()
answer = chain.invoke({"pruduct": "ChatGPT"})

print(answer)

ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 이해하고 적절한 답변을 생성하여 대화할 수 있는 능력을 가지고 있습니다. 다양한 주제에 대해 정보를 제공하고, 창의적인 글쓰기, 문제 해결 등 여러 용도로 활용될 수 있습니다.


<br>

# <b>4. batch: 배치(단위 실행)</b>

<br>

### 함수 chain.batch는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 
### 각 딕셔너리에 있는 product 키의 값을 사용하여 일괄 처리를 수행합니다. 

<br>

### batch

In [5]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1,
    max_tokens=2048,
)

prompt = PromptTemplate.from_template("{pruduct}에 대해서 3문장으로 설명해줘.")
chain = prompt | llm | StrOutputParser()

# 주어진 product 리스트를 batch 처리하는 함수 호출. 묶어서 그룹 단위로 실행된다.
answer = chain.batch(
    [
        {"pruduct": "ChatGPT"}, 
     	{"pruduct": "Instagram"}
    ]
)

print(answer)
print(answer[0])

['ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 인간처럼 자연스럽고 유창한 대화를 생성할 수 있습니다. 다양한 주제에 대한 정보 제공, 문제 해결, 창의적인 글쓰기 등 여러 용도로 활용될 수 있습니다.', '인스타그램은 사용자가 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자들이 창의적으로 콘텐츠를 꾸밀 수 있도록 돕습니다. 또한, 친구와의 소통, 유명인 및 브랜드의 팔로우를 통해 다양한 사람들과 연결될 수 있는 공간입니다.']
ChatGPT는 OpenAI에서 개발한 대화형 인공지능 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 인간처럼 자연스럽고 유창한 대화를 생성할 수 있습니다. 다양한 주제에 대한 정보 제공, 문제 해결, 창의적인 글쓰기 등 여러 용도로 활용될 수 있습니다.


<br>

### max_concurrency 매개변수를 사용하여 동시 요청 수를 설정할 수 있습니다
### config 딕셔너리는 max_concurrency 키를 통해 동시에 처리할 수 있는 최대 작업 수를 설정합니다. 
### 여기서는 최대 3개의 작업을 동시에 처리하도록 설정되어 있습니다.

<br>

### batch

In [6]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1,
    max_tokens=2048,
)

prompt = PromptTemplate.from_template("{pruduct}에 대해서 3문장으로 설명해줘.")
chain = prompt | llm | StrOutputParser()

answer = chain.batch(
    [
        {"pruduct": "ChatGPT"},
        {"pruduct": "Instagram"},
        {"pruduct": "멀티모달"},
        {"pruduct": "프로그래밍"},
        {"pruduct": "머신러닝"},
    ],
    config={"max_concurrency": 3},
)

print(answer)

['ChatGPT는 OpenAI에서 개발한 인공지능 언어 모델로, 자연어 처리 기술을 기반으로 합니다. 사용자의 질문이나 요청에 대해 대화 형식으로 응답하며, 다양한 주제에 대한 정보를 제공할 수 있습니다. 이 모델은 대규모 데이터셋을 학습하여 인간과 유사한 방식으로 언어를 이해하고 생성하는 능력을 갖추고 있습니다.', '인스타그램은 사용자가 사진과 동영상을 공유할 수 있는 소셜 미디어 플랫폼입니다. 다양한 필터와 편집 도구를 제공하여 사용자들이 창의적으로 콘텐츠를 꾸밀 수 있도록 돕습니다. 또한, 친구와의 소통, 유명인 및 브랜드와의 연결을 통해 전 세계적으로 인기를 끌고 있습니다.', '멀티모달은 다양한 형태의 데이터를 동시에 처리하고 분석하는 접근 방식을 의미합니다. 예를 들어, 텍스트, 이미지, 오디오 등 서로 다른 유형의 정보를 결합하여 보다 풍부하고 정확한 인사이트를 도출할 수 있습니다. 이러한 기술은 인공지능, 머신러닝, 자연어 처리 등 여러 분야에서 활용되어, 사용자 경험을 향상시키고 복잡한 문제를 해결하는 데 기여합니다.', '프로그래밍은 컴퓨터가 수행할 작업을 정의하는 과정으로, 특정 언어를 사용하여 코드로 작성됩니다. 이 과정에서는 알고리즘을 설계하고, 문제를 해결하기 위한 논리를 구현하게 됩니다. 결과적으로 프로그래밍은 소프트웨어 개발, 웹 애플리케이션, 게임 등 다양한 분야에서 활용됩니다.', '머신러닝은 컴퓨터가 데이터에서 패턴을 학습하고 예측을 수행할 수 있도록 하는 인공지능의 한 분야입니다. 이 과정에서는 알고리즘을 사용하여 입력 데이터로부터 모델을 생성하고, 이를 통해 새로운 데이터에 대한 결정을 내립니다. 머신러닝은 이미지 인식, 자연어 처리, 추천 시스템 등 다양한 응용 분야에서 활용되고 있습니다.']


<br>

# <b>5. async stream: 비동기 스트림</b>

<br>

# <b>5.1 비동기 메소드?</b>

<br>

### 비동기 메소드는 프로그램이 특정 작업을 수행하고 있을 때, 
### 그 작업이 완료될 때까지 프로그램 전체를 멈추지 않고 다른 작업을 계속 수행할 수 있게 해주는 방법입니다. 
### 즉, 작업의 완료를 기다리는 동안 다른 코드의 실행이 가능합니다.

<br>

# <b>5.2 일상의 예: 커피숍</b>

<br>

### 비동기 메소드를 커피숍에서의 상황으로 비유해보겠습니다. 카페에 가서 커피를 주문했다고 상상해 봅시다.

<br>

### - 동기 방식(Synchronous): <br>
#### &nbsp;&nbsp;&nbsp;&nbsp;커피를 주문한 후, 커피가 나올 때까지 그 자리에서 기다립니다. 
#### &nbsp;&nbsp;&nbsp;&nbsp;기다리는 동안 다른 어떠한 일을 하지 못하고 오로지 커피만을 기다리는 상태입니다.

### - 비동기 방식(Asynchronous): <br> 
#### &nbsp;&nbsp;&nbsp;&nbsp; 커피를 주문한 후, 기다리는 동안에 다른 일을 하는 거예요. (예: 책 읽기, 친구와 대화하기)을 합니다. 
#### &nbsp;&nbsp;&nbsp;&nbsp; 커피가 준비되면 번호가 호출되고, 이때 커피를 받으러 갑니다.

<br>

# <b>5.3 프로그래밍에서의 예</b>

<br>

### 웹 서버를 개발할 때 비동기 메소드는 매우 유용하게 사용됩니다. 
### 예를 들어, 사용자가 데이터베이스에서 정보를 요청하는 상황을 생각해 볼 수 있습니다.

<br>

### - 동기 방식: 

##### &nbsp;&nbsp;&nbsp;&nbsp; 서버는 데이터베이스의 응답을 기다리면서 다른 요청을 처리하지 못하고 멈춰 있습니다. 
##### &nbsp;&nbsp;&nbsp;&nbsp; 이는 효율성이 떨어질 수 있습니다.

### - 비동기 방식:

##### &nbsp;&nbsp;&nbsp;&nbsp; 서버는 데이터를 요청하고, 그 응답을 기다리는 동안 다른 사용자의 요청을 계속 처리할 수 있습니다. 
##### &nbsp;&nbsp;&nbsp;&nbsp; 데이터베이스에서 응답이 오면, 그때 해당 작업을 완료합니다.

<br>

# <b>6. async stream: 비동기 스트림</b>

<br>

### chain.astream() 함수는 비동기 스트림을 생성하며, 주어진 product에 대한 메시지를 비동기적으로 처리합니다.

<br>

### 비동기 for 루프(async for)를 사용하여 스트림에서 메시지를 순차 적으로 받아오고, 
### print() 함수를 통해 메시지의 내용(s.content)을 즉시 출력합니다. 

### end=""는 출력 후 줄 바꿈을 하지 않도록 설정하며, flush=True는 출력 버퍼를 강제로 비워 즉시 출력되도록 합니다.

<br>